# Memory Graph Preview

Let's see how a memory graph can enhance our CRM system by tracking:
- Questions asked
- Answers provided
- Customer feedback
- Support context

In [ ]:
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os
from datetime import datetime

load_dotenv()
driver = GraphDatabase.driver(
    os.getenv('NEO4J_URI'),
    auth=(os.getenv('NEO4J_USERNAME'), os.getenv('NEO4J_PASSWORD'))
)

## Record Customer Question

In [ ]:
def record_interaction(customer_id, question):
    with driver.session() as session:
        result = session.run("""
        // Create interaction nodes
        MATCH (c:Customer {customer_id: $customer_id})
        CREATE (i:Interaction {
            id: randomUUID(),
            timestamp: datetime(),
            question: $question
        })
        CREATE (c)-[:HAD_INTERACTION]->(i)
        
        // Get context from recent interactions
        WITH c, i
        MATCH (c)-[:HAD_INTERACTION]->(prev:Interaction)
        WHERE prev.timestamp > datetime() - duration('P7D')
        
        // Create memory nodes
        CREATE (m:Memory {
            id: randomUUID(),
            type: 'context',
            content: 'Previous interaction: ' + prev.question
        })
        CREATE (i)-[:HAS_CONTEXT]->(m)
        
        RETURN i.id as interaction_id
        """, customer_id=customer_id, question=question)
        
        return result.single()['interaction_id']

# Record a new question
interaction_id = record_interaction('C1', 'How do I extend my laptop battery life?')
print(f"Recorded interaction: {interaction_id}")

## Analyze Interaction History

In [ ]:
def get_interaction_history(customer_id):
    with driver.session() as session:
        result = session.run("""
        MATCH (c:Customer {customer_id: $customer_id})
        MATCH (c)-[:HAD_INTERACTION]->(i:Interaction)
        OPTIONAL MATCH (i)-[:HAS_CONTEXT]->(m:Memory)
        RETURN i.timestamp as timestamp,
               i.question as question,
               collect(m.content) as context
        ORDER BY i.timestamp DESC
        """, customer_id=customer_id)
        
        return list(result)

# View interaction history
history = get_interaction_history('C1')
for interaction in history:
    print(f"\nTime: {interaction['timestamp']}")
    print(f"Question: {interaction['question']}")
    if interaction['context']:
        print("Context:")
        for ctx in interaction['context']:
            print(f"- {ctx}")

## Quick Challenge

Use both GraphRAG patterns and the memory graph to:
1. Find relevant product info
2. Check interaction history
3. Provide a contextual answer

In [ ]:
def answer_with_context(customer_id, question):
    # Your solution here!
    pass